In [2]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
rating=pd.read_excel("toy_dataset1.xlsx",index_col=0)
rating

,action1,action2,action3,romantic1,romantic2,romantic3
user 1,4,5,3,0,2,1
user 2,5,3,3,2,2,0
user 3,1,0,0,4,5,4
user 4,0,2,1,4,0,3
user 5,1,0,2,3,3,4


In [8]:
# rating zero is issue bxse a user can't give zero rating so we will change this into 
# 

In [9]:
def standardize(row):
    new_row=(row-row.mean())/(row.max()-row.min())
    return new_row

rating_std=rating.apply(standardize)
rating_std

,action1,action2,action3,romantic1,romantic2,romantic3
user 1,0.36,0.6,0.400000,-0.65,-0.08,-0.35
user 2,0.56,0.2,0.400000,-0.15,-0.08,-0.60
user 3,-0.24,-0.4,-0.600000,0.35,0.52,0.40
user 4,-0.44,0.0,-0.266667,0.35,-0.48,0.15
user 5,-0.24,-0.4,0.066667,0.10,0.12,0.40


In [10]:
# now our main moto is to find the similarity in b/w users
# here we will use anguler distance for find similarity b/w two user's

# here we want to implement cosine similarity on column and cosine is apllied on row 
# but row contain user info so we transpose our standerd matrix using (.T)

item_similarity=cosine_similarity(rating_std.T)
item_similarity
# this matrix will be a model for our new user to recomand movie

array([[ 1.        ,  0.70668875,  0.81368151, -0.79941088, -0.02539184,
        -0.91410609],
       [ 0.70668875,  1.        ,  0.72310153, -0.84515425, -0.5189993 ,
        -0.84337386],
       [ 0.81368151,  0.72310153,  1.        , -0.84794611, -0.3799803 ,
        -0.80218063],
       [-0.79941088, -0.84515425, -0.84794611,  1.        ,  0.14803913,
         0.72374686],
       [-0.02539184, -0.5189993 , -0.3799803 ,  0.14803913,  1.        ,
         0.39393939],
       [-0.91410609, -0.84337386, -0.80218063,  0.72374686,  0.39393939,
         1.        ]])

In [11]:
# here we convert this array into dataframe so in future that will be easy to use 
# for recomandation
item_similarity_df=pd.DataFrame(item_similarity,index=rating.columns,columns=rating.columns)
item_similarity_df

,action1,action2,action3,romantic1,romantic2,romantic3
action1,1.000000,0.706689,0.813682,-0.799411,-0.025392,-0.914106
action2,0.706689,1.000000,0.723102,-0.845154,-0.518999,-0.843374
action3,0.813682,0.723102,1.000000,-0.847946,-0.379980,-0.802181
romantic1,-0.799411,-0.845154,-0.847946,1.000000,0.148039,0.723747
romantic2,-0.025392,-0.518999,-0.379980,0.148039,1.000000,0.393939
romantic3,-0.914106,-0.843374,-0.802181,0.723747,0.393939,1.000000


In [13]:
# above data we can understand in precentage like 
# action1 and action1 are similar 100%
# action1 and action2 are simillar 70%

# now we will predict/ recommand based on rating


In [17]:
# let's make recommandations
# and this function will return the similarity score with other movie

def get_similar_movies(movie_name,user_rating):
    similar_score=item_similarity_df[movie_name]*(user_rating-2.5)
#     2.5 is mid value of user rationg out of 5
# from this subtraction we got correct recommandation 
    similar_score=similar_score.sort_values(ascending=False)
    
    return similar_score

# here first three row recommand similar rating movie and after three similar movie recommand
# based on user rating 
print(get_similar_movies('action1',5))
    

action1      2.500000
action3      2.034204
action2      1.766722
romantic2   -0.063480
romantic1   -1.998527
romantic3   -2.285265
Name: action1, dtype: float64


In [18]:
print(get_similar_movies('romantic3',1))

action1      1.371159
action2      1.265061
action3      1.203271
romantic2   -0.590909
romantic1   -1.085620
romantic3   -1.500000
Name: romantic3, dtype: float64


In [19]:
# above done for one movie what if user rated sevral movie'

In [24]:
action_lover=[("action1",5),("romantic2",1),("romantic3",1)] #user input/users info
similar_movie=pd.DataFrame()

for movie,rating in action_lover:
    similar_movie=similar_movie.append(get_similar_movies(movie,rating),ignore_index=True)

    
similar_movie.head()
similar_movie.sum().sort_values(ascending=False)
    

action1      3.909247
action2      3.810282
action3      3.807445
romantic2   -2.154389
romantic1   -3.306206
romantic3   -4.376174
dtype: float64